## Part II: `Candidate` Extraction

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the `Corpus`

First, we will load the `Corpus` that we preprocessed in Part I:

In [3]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Training').one()
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware Training) contains 50 Documents


## Defining a `Candidate` Schema

In [4]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

## Writing a basic `CandidateExtractor`

In [5]:
from snorkel.candidates import OmniNgrams

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)

In [6]:
from hardware_utils import load_extended_parts_dict

filename='data/hardware/hardware_gold.csv'
parts_dict = load_extended_parts_dict(filename)
print "Loaded %d part numbers." % len(parts_dict)

Loaded 5331 part numbers.


In [7]:
from snorkel.matchers import DictionaryMatch, RegexMatchSpan

parts_matcher = DictionaryMatch(d=parts_dict, ignore_case=True)
temp_matcher = RegexMatchSpan(rgx=ur'-\s?[5-7][05]')

In [8]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part_Temp, [omni_ngrams, omni_ngrams], [parts_matcher, temp_matcher])

## Running the `CandidateExtractor`

In [9]:
%time candidates = ce.extract(corpus.documents, 'Hardware Training Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: candidate_set.name [SQL: u'INSERT INTO candidate_set (name) VALUES (?)'] [parameters: ('Hardware Training Candidates',)]

In [ ]:
for c in candidates[:5]:
    print c

In [ ]:
# for i in range(len(candidates)):
#     c = candidates[i]
#     if c[0].parent.id == 1518 and c[1].parent.id == 95674:
#         print c
# print len(candidates)
# print len([c.id for c in candidates])
# print len([c for c in candidates])
# print candidates.candidates.count()
# print type(candidates)
# foo = []
# for i in range(len(candidates)): 
#     foo.append(candidates[i].id)
# print len(foo)
# print foo

In [ ]:
# phrase_ids = set([])
# for doc in corpus.documents:
#     for phrase in doc.phrases:
#         if phrase.id in phrase_ids:
#             import pdb; pdb.set_trace()
#         else:
#             phrase_ids.add(phrase.id)

In [ ]:
# print candidates[111][0].parent
# print candidates[112][0].parent

### Saving the extracted candidates

In [ ]:
session.add(candidates)
session.commit()

### Reloading the candidates

In [ ]:
from snorkel.models import CandidateSet

candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
print "%s contains %d Candidates" % (candidates, len(candidates))

### Repeating for development and test corpora

In [ ]:
for corpus_name in ['Hardware Development', 'Hardware Test']:
    corpus = session.query(Corpus).filter(Corpus.name == corpus_name).one()
    %time candidates = ce.extract(corpus.documents, corpus_name + ' Candidates', session)
    session.add(c)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

In [ ]:
# If necessary
# import os
# os.system('cp snorkel.db snorkel.db\ candidates');

Next, in Part 3, we will load `Labels` for each of our `Candidates` so that we can evaluate performance.